[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobertTLange/mle-logging/blob/main/examples/02_advanced.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

from mle_logging import MLELogger

# Log Different Random Seeds for Same Configuration

If you provide a .json file path and a seed_id, the log will be created in a sub-directory.

Furthermore, the .json file will be copied for reproducibility.

Multiple simultanous runs (different seeds) can now log to the same directory. Everything else remains the same.

In [2]:
# Instantiate logging to experiment_dir for two random seeds
log_seed_1 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                       what_to_track = ['train_loss', 'test_loss'],
                       experiment_dir = "multi_seed_dir/",
                       config_fname = "config_1.json",     # Provide path to .json config
                       seed_id = "seed_1")                 # Provide some seed identifier (str)   

log_seed_2 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                       what_to_track = ['train_loss', 'test_loss'],
                       experiment_dir = "multi_seed_dir/",
                       config_fname = "config_1.json",     # Provide path to .json config
                       seed_id = "seed_2")                 # Provide some seed identifier (str)   

In [3]:
# Save some time series statistics
time_tic = {'num_updates': 10,
            'num_epochs': 1}
stats_tic = {'train_loss': 0.1234,
             'test_loss': 0.1235}

# Update the log with collected data & save them to .hdf5
log_seed_1.update(time_tic, stats_tic, save=True)
log_seed_2.update(time_tic, stats_tic, save=True)

In [4]:
import os, datetime
from mle_logging import merge_seed_logs, load_log

timestr = datetime.datetime.today().strftime("%Y-%m-%d")[2:]
experiment_dir = f"multi_seed_dir/{timestr}_config_1/"
merged_path = os.path.join(experiment_dir, "logs", "seed_aggregated.hdf5")

# Merge different random seeds into one .hdf5 file
merge_seed_logs(merged_path, experiment_dir)

# Load the merged log - Individual seeds can be accessed via log.seed_1, etc.
log = load_log(experiment_dir)
log.eval_ids, log.seed_1.stats.train_loss

(['seed_1', 'seed_2'], array([0.1234], dtype=float32))

In [5]:
# Load the merged log and aggregate over random seeds (compute stats)
log = load_log(experiment_dir, aggregate_seeds=True)
log.eval_ids, log.stats.train_loss.keys()

Only single aggregated configuration or random seed loaded.


(None, odict_keys(['mean', 'std', 'p50', 'p10', 'p25', 'p75', 'p90']))

## Log Different Configurations with Different Random Seeds

In [6]:
# Instantiate logging to experiment_dir for two .json configurations and two seeds
log_c1_s1 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                      what_to_track = ['train_loss', 'test_loss'],
                      experiment_dir = "multi_config_dir/",
                      config_fname = "config_1.json",     # Provide path to .json config
                      seed_id = "seed_1")                 # Provide some seed identifier (str)   

log_c1_s2 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                      what_to_track = ['train_loss', 'test_loss'],
                      experiment_dir = "multi_config_dir/",
                      config_fname = "config_1.json",     # Provide path to .json config
                      seed_id = "seed_2")                 # Provide some seed identifier (str)   

log_c2_s1 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                      what_to_track = ['train_loss', 'test_loss'],
                      experiment_dir = "multi_config_dir/",
                      config_fname = "config_2.json",     # Provide path to .json config
                      seed_id = "seed_1")                 # Provide some seed identifier (str)   

log_c2_s2 = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                      what_to_track = ['train_loss', 'test_loss'],
                      experiment_dir = "multi_config_dir/",
                      config_fname = "config_2.json",     # Provide path to .json config
                      seed_id = "seed_2")                 # Provide some seed identifier (str)     

In [7]:
# Update the log with collected data & save them to .hdf5
log_c1_s1.update(time_tic, stats_tic, save=True)
log_c1_s2.update(time_tic, stats_tic, save=True)
log_c2_s1.update(time_tic, stats_tic, save=True)
log_c2_s2.update(time_tic, stats_tic, save=True)

In [8]:
# Merge different random seeds for each config into separate .hdf5 file
merge_seed_logs(f"multi_config_dir/{timestr}_config_1/logs/config_1.hdf5", 
                f"multi_config_dir/{timestr}_config_1/")
merge_seed_logs(f"multi_config_dir/{timestr}_config_2/logs/config_2.hdf5", 
                f"multi_config_dir/{timestr}_config_2/")

In [9]:
# Aggregate the different merged configuration .hdf5 files into single meta log
from mle_logging import merge_config_logs
merge_config_logs(experiment_dir = "multi_config_dir/",
                  all_run_ids = ["config_1", "config_2"])

In [10]:
# Afterwards load in the meta log object
from mle_logging import load_meta_log
meta_log = load_meta_log("multi_config_dir/meta_log.hdf5")
meta_log.eval_ids, meta_log.config_1.stats.test_loss.keys()

(['config_2', 'config_1'],
 odict_keys(['mean', 'std', 'p50', 'p10', 'p25', 'p75', 'p90']))

In [11]:
meta_log = load_meta_log("multi_config_dir/meta_log.hdf5", aggregate_seeds=False)
meta_log.eval_ids

['config_1_seed_1', 'config_1_seed_2', 'config_2_seed_1', 'config_2_seed_2']

# Logging Every k-th Checkpoint Update

In [12]:
# Instantiate logging to experiment_dir
import numpy as np
model = {"layer_1": {"w": np.random.normal(0, 1, (784, 512)),
                     "b": np.random.normal(0, 1, (512,))}}

log = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                what_to_track = ['train_loss', 'test_loss'],
                experiment_dir = 'every_k_dir/',
                model_type = 'jax',
                ckpt_time_to_track = 'num_updates',
                save_every_k_ckpt = 2)

In [13]:
time_tic = {'num_updates': 10, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 20, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 30, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 40, 'num_epochs': 1}
log.update(time_tic, stats_tic, model, save=True)

log.every_k_ckpt_list, log.every_k_storage_time

(['every_k_dir/models/every_k/21-08-06_no_seed_provided_k_1.pkl',
  'every_k_dir/models/every_k/21-08-06_no_seed_provided_k_3.pkl'],
 [10, 30])

# Logging Top-k Checkpoints Based on Metric

In [14]:
# Instantiate logging to experiment_dir
log = MLELogger(time_to_track = ['num_updates', 'num_epochs'],
                what_to_track = ['train_loss', 'test_loss'],
                experiment_dir = "top_k_dir/",
                model_type = 'jax',
                ckpt_time_to_track = 'num_updates',
                save_top_k_ckpt = 2,
                top_k_metric_name = "test_loss",
                top_k_minimize_metric = True)

In [15]:
time_tic = {'num_updates': 10, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.1235}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 20, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.11}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 30, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.09}
log.update(time_tic, stats_tic, model, save=True)

time_tic = {'num_updates': 40, 'num_epochs': 1}
stats_tic = {'train_loss': 0.1234, 'test_loss': 0.12}
log.update(time_tic, stats_tic, model, save=True)

log.top_k_performance, log.top_k_storage_time

([0.09, 0.11], [30, 20])